In [12]:
busy = [
{'start' : '10:30',
'stop' : '10:50'
},
{'start' : '18:40',
'stop' : '18:50'
},
{'start' : '14:40',
'stop' : '15:50'
},
{'start' : '16:40',
'stop' : '17:20'
},
{'start' : '20:05',
'stop' : '20:20'
}
]

from datetime import datetime
from datetime import time, timedelta
from dataclasses import dataclass


@dataclass
class TimeWindow:
    start: time
    end: time

    def __str__(self):
        return f"Окно с {self.start.strftime('%H:%M')} до {self.end.strftime('%H:%M')}"


class Schedule:

    def __init__(self,
        working_day_start: str = '9:00',
        working_day_end: str = '21:00'
    ):
        self.working_day = TimeWindow(
            start=self._str2time(working_day_start),
            end=self._str2time(working_day_end)
        )
        self.half_hour_windows: list[TimeWindow] = []


    def half_hour_windows_from_busy_time(self, busy):
        busy_time = []
        for period in busy:
            busy_time.append(
                TimeWindow(start=self._str2time(period['start']), end=self._str2time(period['stop']))
            )
        busy_time.sort(key=lambda period: period.start)
        cursor = self.working_day.start
        while cursor < self.working_day.end:
            next_half_hour = self.add_minutes_to_time(cursor, 30)
            if not busy_time or next_half_hour <= busy_time[0].start:
                if next_half_hour > self.working_day.end:
                    break
                self.half_hour_windows.append(
                    TimeWindow(start=cursor, end=next_half_hour)
                )
                cursor = next_half_hour
            else:
                cursor = busy_time.pop(0).end

    
    @staticmethod
    def _str2time(str_time):
        try:
            return datetime.strptime(str_time, '%H:%M').time()
        except ValueError:
            raise ValueError(f"Incorrect time format for {str_time}, expected '%H:%M'")

    @staticmethod
    def add_minutes_to_time(original_time, minutes):
        combined_datetime = datetime.combine(datetime.today(), original_time)

        new_datetime = combined_datetime + timedelta(minutes=minutes)

        return new_datetime.time()


dec = Schedule()
dec.half_hour_windows_from_busy_time(busy)
dec.half_hour_windows


[TimeWindow(start=datetime.time(9, 0), end=datetime.time(9, 30)),
 TimeWindow(start=datetime.time(9, 30), end=datetime.time(10, 0)),
 TimeWindow(start=datetime.time(10, 0), end=datetime.time(10, 30)),
 TimeWindow(start=datetime.time(10, 50), end=datetime.time(11, 20)),
 TimeWindow(start=datetime.time(11, 20), end=datetime.time(11, 50)),
 TimeWindow(start=datetime.time(11, 50), end=datetime.time(12, 20)),
 TimeWindow(start=datetime.time(12, 20), end=datetime.time(12, 50)),
 TimeWindow(start=datetime.time(12, 50), end=datetime.time(13, 20)),
 TimeWindow(start=datetime.time(13, 20), end=datetime.time(13, 50)),
 TimeWindow(start=datetime.time(13, 50), end=datetime.time(14, 20)),
 TimeWindow(start=datetime.time(15, 50), end=datetime.time(16, 20)),
 TimeWindow(start=datetime.time(17, 20), end=datetime.time(17, 50)),
 TimeWindow(start=datetime.time(17, 50), end=datetime.time(18, 20)),
 TimeWindow(start=datetime.time(18, 50), end=datetime.time(19, 20)),
 TimeWindow(start=datetime.time(19, 20),

In [43]:
try:
    print(datetime.strptime('20:', '%H:%M').time())
except ValueError:
    raise ValueError(f"Incorrect time format for 1, expected '%H:%M'")

ValueError: Incorrect time format for 1, expected '%H:%M'

In [47]:
a = TimeWindow(datetime.strptime("20:20", "%H:%M").time(), datetime.strptime("20:20", "%H:%M").time())
str(a)

'Окно с 20:20 до 20:20'

In [58]:
timedelta(a.start, a.end)

TypeError: unsupported type for timedelta seconds component: datetime.time

In [59]:
a = [1, 3, 2]
a.sort()
a.pop(0)

1